"Live data" exported from research fish:

In research fish, go to:
* eVal
* "Export" tab

Then enter:
* Submission Period: Live Data

With each of the possible Export Types selected:
* Download Export
* Wait for download to be ready in "My downloads" tab.
* Save files in data/ResearchFish/raw directory.

In [1]:
import numpy as np
import pandas as pd
import glob
import MatchIDs

In [2]:
# get list of files in ResearchFish directory
BASE_DIR = 'data/ResearchFish/raw'
files = glob.glob(BASE_DIR+'/*.xlsx')

# take second word of file name to be the category/type of table
categories = [file.split()[1] for file in files]

In [3]:
if 'rht' in categories:
    i = categories.index('rht')
    rht = pd.read_excel(files[i])
    rht.set_index('Award Reference',inplace=True)
    
    # remove test entries
    rht = rht[~rht.index.str.contains('Test')]

    print('Loaded rht (award details), has shape',
          rht.shape)
       
    files.remove(files[i])
    categories.remove('rht')
    
    
else:
    raise Exception('Expected a file of general award details containing the string rht, but didnt find it.')


Loaded rht (award details), has shape (286, 49)


In [4]:
# load all other research fish tables
data = {}

for i,file in enumerate(files):
    df_tmp = pd.read_excel(file)    

    # some exported spreadsheets seem to have duplicated entries - delete them
    # for those with an Entry ID, don't consider that column as it's always
    # unique.
    if df_tmp.columns.contains('Entry ID'):
        df_tmp.drop_duplicates(subset=df_tmp.columns.drop('Entry ID'),
                              inplace=True)
    else:
        df_tmp.drop_duplicates(inplace=True)
            
    unique_refs = df_tmp['Award Reference'].unique()  
    refs_in_rht = sum(rht.index.isin(unique_refs))
    
    data[categories[i]] = df_tmp

    print(categories[i],'has',len(unique_refs),'unique awards, of which',
          refs_in_rht,'in rht.')


meetings-conferences has 261 unique awards, of which 259 in rht.
facilities has 11 unique awards, of which 11 in rht.
technical-products has 7 unique awards, of which 7 in rht.
meetings_conferences_conference has 168 unique awards, of which 168 in rht.
creative-products has 4 unique awards, of which 4 in rht.
ffs-compulsory-narrative-report has 266 unique awards, of which 265 in rht.
products-or-interventions has 13 unique awards, of which 13 in rht.
research-database-models has 23 unique awards, of which 23 in rht.
publications has 165 unique awards, of which 165 in rht.
hrcs has 221 unique awards, of which 219 in rht.
influence-on-policy-and-practice has 23 unique awards, of which 23 in rht.
intellectual-property-and-licensing has 8 unique awards, of which 8 in rht.
research-materials has 49 unique awards, of which 49 in rht.
next-destination-and-recruitment has 58 unique awards, of which 58 in rht.
sf2 has 111 unique awards, of which 111 in rht.
awards-and-recognition has 82 unique 

In [5]:
# create a column in rht with counts of entries in other tables
for key,df in data.items():
    n_entries = df.groupby('Award Reference')['Award Reference'].count()
    n_entries.name = key
    rht[key] = n_entries
    rht[key].fillna(0,inplace=True)
    
    print(key, 'has an award with',n_entries.max(),'entries')

meetings-conferences has an award with 1 entries
facilities has an award with 2 entries
technical-products has an award with 3 entries
meetings_conferences_conference has an award with 27 entries
creative-products has an award with 1 entries
ffs-compulsory-narrative-report has an award with 2 entries
products-or-interventions has an award with 2 entries
research-database-models has an award with 2 entries
publications has an award with 42 entries
hrcs has an award with 4 entries
influence-on-policy-and-practice has an award with 5 entries
intellectual-property-and-licensing has an award with 2 entries
research-materials has an award with 4 entries
next-destination-and-recruitment has an award with 10 entries
sf2 has an award with 24 entries
awards-and-recognition has an award with 25 entries
other-outputs-and-knowledge has an award with 6 entries
dissemination-to-non-academic-audiences has an award with 52 entries
spin-outs has an award with 1 entries
collaborations-and-partnerships ha

## Matching IDs to Award Holders Master.xlsx

In [6]:
rht = MatchIDs.find_matches_in_master(rht)
rht.head()

Found 283 matches in master.
3 grants were not found in master:
['1371/72', '1655', '1933']


,Funder ID,Funder,Agreement ID,Linked Agreement,Award Type,Title,RO Location ID,RO,Centre Location ID,Centre,...,research-materials,next-destination-and-recruitment,sf2,awards-and-recognition,other-outputs-and-knowledge,dissemination-to-non-academic-audiences,spin-outs,collaborations-and-partnerships,further-funding-for-your-research-group,MasterID
Award Reference,,,,,,,,,,,,,,,,,,,,,
1405/06,FLFS-,Fight for Sight,FLFS-1405/06,NaN,NaN,Early and Late Ocular Toxicity following Radio...,X00000637,University Hospitals Birmingham NHS Foundation...,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1405/1406
1554/55,FLFS-,Fight for Sight,FLFS-1554/55,NaN,NaN,Role of Apelin/APJ in diabetic retinopathy.,X00000345,University of Manchester,NaN,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1554/1555
1463/64,FLFS-,Fight for Sight,FLFS-1463/64,NaN,Fight for Sight Small Grant Award,Evaluation of the Macular Integrity Assessment...,X00000042,Cardiff University,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,1463/1464
1560/61,FLFS-,Fight for Sight,FLFS-1560/61,NaN,NaN,Mechanisms of retinal ganglion cell death in d...,X00000312,University of Birmingham,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1560/1561
5063/ 64,FLFS-,Fight for Sight,FLFS-5063/ 64,NaN,NaN,Characterisation of novel neuronal receptors a...,X00000347,University of Nottingham,NaN,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5063/5064


In [7]:
# add master id to the other tables
for key,df in data.items():
    data[key] = df.join(rht['MasterID'],on='Award Reference')
    print(data[key]['MasterID'].nunique(),'unique master ids created in',key)

257 unique master ids created in meetings-conferences
11 unique master ids created in facilities
7 unique master ids created in technical-products
166 unique master ids created in meetings_conferences_conference
4 unique master ids created in creative-products
262 unique master ids created in ffs-compulsory-narrative-report
13 unique master ids created in products-or-interventions
23 unique master ids created in research-database-models
163 unique master ids created in publications
216 unique master ids created in hrcs
23 unique master ids created in influence-on-policy-and-practice
8 unique master ids created in intellectual-property-and-licensing
48 unique master ids created in research-materials
58 unique master ids created in next-destination-and-recruitment
110 unique master ids created in sf2
82 unique master ids created in awards-and-recognition
43 unique master ids created in other-outputs-and-knowledge
112 unique master ids created in dissemination-to-non-academic-audiences
5 

In [8]:
funders = {'Alzheimers Research UK, Cambridge':"Alzheimer's Research UK",
          'Crossley Barnes PhD Studentship award':'Crossley-Barnes Bequest',
          'Crossly Barnes Bequest, University of Liverpool Institute of Ageing and Chronic Disease':'Crossley-Barnes Bequest',
          'Department of Education and Learning NI':'Department for Employment and Learning Northern Ireland (DELNI)',
          'EPSRC and CRUK':'Engineering and Physical Sciences Research Council (EPSRC), Cancer Research UK',
          'Fight For Sight':'Fight for Sight',
          'Fight for Sight and ARUK':'Fight for Sight, Arthritis Research UK',
          'HPSS R and D Office and MRC':'HPSS R and D Office, Medical Research Council (MRC)',
          'IGA and UKEGS':'International Glaucoma Association (IGA), UK and Eire Glaucoma Society (UKEGS)',
          'Institute of Ageing and Chronic Disease, Faculty of Health & Life Sciences,  University of Liverpool':'Institute of Ageing and Chronic Disease University of Liverpool',
          'MRC Confidence in Concept Scheme':'Medical Research Council (MRC)',
          'MRC-UK':'Medical Research Council (MRC)',
          'Macular Disease Soc':'Macular Society',
          'Merton College, University of Oxford':'University of Oxford',
          'Moorfield Eye Charity':'Moorfields Eye Charity',
          "National Centre for the Replacement, Refinement and Reduction of Animals in Research (NC3Rs)":'National Centre for the Replacement Refinement and Reduction of Animals in Research (NC3Rs)',
          'NCR3R CrackiT':'National Centre for the Replacement Refinement and Reduction of Animals in Research (NC3Rs)',
          'NIHR BMRC GOSH/ Institute of Child Health UCL':'NIHR BRC Great Ormond Street, UCL Great Ormond Street Institute of Child Health',
          'NIHR Biomedical Research Centre and Biomedical Research Unit for Dementia':'NIHR BRC, NIHR BRU Dementia',
          'NIHR Biomedical Research Centreat Moorfields Eye Hospital and UCL Institute of Ophthalmology':'NIHR BRC Moorfields, UCL Institute of Ophthalmology',
          'NIHR Moorfields Biomedical Research Centre':'NIHR BRC Moorfields',
          'NIHR Newcastle Biomedical Research Centre':'NIHR BRC Newcastle',
          'NIHR/BRC':'NIHR BRC',
          "Queen's University of Belfast":"Queens University",
          'The Humane Research Trust Laboratory - infrastructure support.':'The Humane Research Trust',
          'The Queen Elizabeth Diamond Jubilee Trust':'Queen Elizabeth Diamond Jubilee Trust',
          'The Wellcome Trust Ltd':'Wellcome Trust',
          'UCL Institute of Child Health':'UCL Great Ormond Street Institute of Child Health',
          'University College London (UCL)':'University College London',
          'University of Liverpool/EPSRC Impact Award':'University of Liverpool, Engineering and Physical Sciences Research Council (EPSRC)',
          'Wellcome Trust Institutional Strategic Support Fund':'Wellcome Trust',
          'Wellcome Trust Pathfinder':'Wellcome Trust',
          'Wellcome Trust Senior Investigator Award':'Wellcome Trust',
          'National Institutes of Health Research':'National Institute for Health Research (NIHR)'}

print(data['further-funding-for-your-research-group']['Parent Org*'].nunique())

data['further-funding-for-your-research-group']['Parent Org*'].replace(funders,inplace=True)

print(data['further-funding-for-your-research-group']['Parent Org*'].nunique())

128
109


In [10]:
drop_cols = ['Funder ID', 'Funder', 'Agreement ID', 'Linked Agreement',
             'Centre Location ID', 'Centre', 'Centre Metafunder ID',
             'Centre Metafunder', 'Funding Currency', 'Converted Funding Value',
             'Converted Currency', 'PI ORCID', 'Response Code',
             'Number of Invitations', 'PI Last Accessed', 
             'Most Recent Submission Period Entered',
             'Submission Period Start Date', 'Submission Period End Date',
             'Last Submitted Date','Lay Summary','AMRC Animals',
             'AMRC Animals Species','AMRC Animals Modified','AMRC Funder Comments',
             'Reporting Node','Record Created Date','Record Updated Date',
             'Sharing Status','Accepted Invitation?','Currently in Submission Period?',
             'Submitted in most Recent Submission Period?']

rht = rht.drop(drop_cols,axis=1)

In [11]:
# save master
rht = rht.reset_index().set_index('MasterID')
rht.to_excel('data/ResearchFish/ResearchFish_master.xlsx')

In [14]:
for key,df in data.items():
    # save columns not present in master in other files
    #columns_to_keep = [col for col in data[key].columns if col not in rht.columns]
    #columns_to_keep.append('MasterID')
    #columns_to_keep = [col for col in columns_to_keep if col not in ['Award Reporting Period ID','Award Submission Date','Entry ID','Entry Insert Date']]
    #data[key][columns_to_keep].to_excel('data/ResearchFish/ResearchFish_'+key+'.xlsx',index=False)
    
    data[key] = data[key].drop([col for col in drop_cols if col in data[key].columns],axis=1)
    data[key].to_excel('data/ResearchFish/ResearchFish_'+key+'.xlsx',index=False)

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:910: UserWarning: Ignoring URL 'https://www.eventbrite.co.uk/e/new-drugs-for-glaucoma-its-all-about-the-electricity-tickets-28833426555?ref=enivte001&invite=MTA5NzQ2MDgvam9hbm5hQGZpZ2h0Zm9yc2lnaHQub3JnLnVrLzA%3D&utm_source=eb_email&utm_medium=email&utm_campaign=invitemodernv2&ref=enivte001&utm_term=attend' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
